In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import os
print(os.getpid())
%cd ../../

import pylib.mix as mix
import pylib.measurement as mse

path_qucf_ = "../QuCF/tests/qsp-hamiltonian/hermitian"

In [ ]:
for i in range(10):
    plt.close()

In [ ]:
# -----------------------------------------------------------------
# --- i d y1(t) / d t =    a1 * y1 - 1j*b * y2 ---
# --- i d y2(t) / d t =  1j*b * y1 +   a2 * y2 ---
# -----------------------------------------------------------------
from scipy.integrate import RK45
mix.reload_module(mix)

nt = 10
t_max = 5

a1 = 1.
a2 = 1.
b  = 2.

Nt = 1 << nt
t = np.linspace(0, t_max, Nt)
dt = np.diff(t)[0]

# *** Form the Hamiltonian for the system ***
H_orig = np.ones((2, 2), dtype=complex)
H_orig[0,0] = a1

H_orig[0,1] = b
H_orig[1,0] = b

H_orig[1,1] = a2

print()
print("*** Original Hamiltonian ***")
mix.print_matrix(H_orig)

# simulated signals:
Ns = 2
y = np.zeros((Nt,Ns), dtype=complex)

# *** Set the initial signal ***
y[0,0] = np.sqrt(0.4)
y[0,1] = np.sqrt(0.6)

# ---------------------------------------------------------------------------------------------------
# --- Compute the time evolution ---
# ---------------------------------------------------------------------------------------------------
def calc_y(t,y):
    y = -1j*H_orig.dot(y) 
    return y

oo = RK45(
    calc_y,
    t[0], 
    y[0,:], t[-1], first_step=dt, max_step=dt
)

Nt_act = 0
oo.step() # skip one time step
while mix.compare_two_strings(oo.status, "running"):
    oo.step()
#     print(oo.t)
    Nt_act += 1
    y[Nt_act,:] = oo.y
print("sum y(t = t_max)**2: {:0.3e}".format(np.sum(np.abs(y[-1,:])**2)))


# ---------------------------------------------------------------------------------------------------
# --- Compute angles for initialization and block-encoding ---
# ---------------------------------------------------------------------------------------------------
print()
print("--- Initial state vector ---")
mix.print_array(y[0,:])

# *** Normalize the Hamiltonian ***
nonsparsity = 1 
if np.abs(b) > 0: 
    nonsparsity = 2
    
rows_sum = np.zeros(Ns)
for ir in range(Ns):
    rows_sum[ir] = np.sqrt(np.sum(np.abs(H_orig[ir,:])**2))
coef_norm_H = np.max(rows_sum)

final_norm_of_H_ = nonsparsity
if coef_norm_H > 1:
    final_norm_of_H_ *= coef_norm_H
    
H = H_orig/final_norm_of_H_

print("\nNormalization factor: {:0.3e}".format(final_norm_of_H_))

print()
print("*** Normalized Hamiltonian ***")
mix.print_matrix(H)

# *** Compute angles ***
ay_init = 2*np.arccos(y.real[0,0])  # Ry(ay_init_y1)*|0>

coef_OF = nonsparsity # take into account the action of the oracle OH
ay_H00, az_H00 = mix.calc_angles_from_a_value(H[0,0] * coef_OF)
ay_H01, az_H01 = mix.calc_angles_from_a_value(H[0,1] * coef_OF)
ay_H10, az_H10 = mix.calc_angles_from_a_value(H[1,0] * coef_OF)
ay_H11, az_H11 = mix.calc_angles_from_a_value(H[1,1] * coef_OF)


print()
print("--- The angle for the initialization circuit ---")
print("ay_init  {:0.12e}".format(ay_init))
    
print()
print("--- The angles for the BE oracle ---")
print("ay_H00  {:0.12e}".format(ay_H00))
if az_H00 is not None:
    print("az_H00  {:0.12e}".format(az_H00))
    
print("ay_H01  {:0.12e}".format(ay_H01))
if az_H01 is not None:
    print("az_H01  {:0.12e}".format(az_H01))
    
print("ay_H10  {:0.12e}".format(ay_H10))
if az_H10 is not None:
    print("az_H10  {:0.12e}".format(az_H10))
    
print("ay_H11  {:0.12e}".format(ay_H11))
if az_H11 is not None:
    print("az_H11  {:0.12e}".format(az_H11))


# ---------------------------------------------------------------------------------------------------
# --- Plot y1(t) and y2(t) ---
# ---------------------------------------------------------------------------------------------------
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t, y.real[:,0], "-b", linewidth = 2, label="Re y1")
ax.plot(t, y.real[:,1], "-r", linewidth = 2, label="Re y2")

ax.plot(t, y.imag[:,0], "--b", linewidth = 2, label="Im y1")
ax.plot(t, y.imag[:,1], "--r", linewidth = 2, label="Im y2")

# plt.xlim([0,2])

plt.xlabel('$y$')
plt.ylabel("$y$")
ax.legend()
plt.grid(True)
plt.show()

In [ ]:
# -----------------------------------------------------------------
# SIMULATIONS: QuCF/QuCF/tests/qsp-hamiltonian/hermitian/
# READ results from QSP simulations
# -----------------------------------------------------------------
mix.reload_module(mse)
def get_qucf(name_pr = "hermitian"):
    om = mse.MeasOracle__(True)
    om.path_ = path_qucf_
    om.pname_ = name_pr
    om.open()
    om.read_qsvt()
    qy = om.read_qsp_ham_results("hamiltonian", 0)
    qt = om.qsp_get_time_grid("hamiltonian", final_norm_of_H_)
    return qy, qt
# ---------------------------------------------------------------------------------------------------

qy_factor, qt = get_qucf()

# --- REAL ---
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t,               y.real[:,0], "-b", linewidth = 2, label="classical y1")
ax.plot(qt,      qy_factor.real[:,0], "b", marker = 'o', markerfacecolor='None', linestyle='None', label="QuCF y1")

ax.plot(t,              y.real[:,1], "-r", linewidth = 2, label="classical y2")
ax.plot(qt,     qy_factor.real[:,1], "r", marker = 'o', markerfacecolor='None', linestyle='None', label="QuCF y2")

plt.xlabel('$t$')
plt.ylabel("$Re y$")
ax.legend()
plt.grid(True)
plt.show()


# --- IMAG ---
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(t,              y.imag[:,0], "-b", linewidth = 2, label="classical y1")
ax.plot(qt,     qy_factor.imag[:,0], "b", marker = 'o', markerfacecolor='None', linestyle='None', label="QuCF y1")

ax.plot(t,              y.imag[:,1], "-r", linewidth = 2, label="classical y2")
ax.plot(qt,     qy_factor.imag[:,1], "r", marker = 'o', markerfacecolor='None', linestyle='None', label="QuCF y2")

plt.xlabel('$t$')
plt.ylabel("$Im y$")
ax.legend()
plt.grid(True)
plt.show()